# DI.fm - Generate Playlists

In [ ]:
!pip install bs4
!pip install requests

In [1]:
import requests, os
from bs4 import BeautifulSoup

`ALL_CHANNELS`: Webscrapping DI.fm is a no-go. But there is this JSON where every channel's currently played song is reported. We are getting the channel names from there.

`CHANNEL_URL`: Each channel URL is formed like this

`KEY`: Create a `di_fm.key` text file where you place your KEY. No line breaks or anything, just the key.

In [2]:
ALL_CHANNELS = "https://api.audioaddict.com/v1/di/currently_playing"  # All channels exist here!
# https://api.audioaddict.com/v1/rockradio/currently_playing
# https://api.audioaddict.com/v1/zenradio/currently_playing
# https://api.audioaddict.com/v1/jazzradio/currently_playing
# https://api.audioaddict.com/v1/classicalradio/currently_playing
# https://api.audioaddict.com/v1/radiotunes/currently_playing
CHANNEL_URL  = "http://api.audioaddict.com/v1/di/listen/premium_high/%s.pls?listen_key=%s"
KEY          = open('di_fm.key', 'r').read()

Let's try to "invent" our own groups, by using keywords. Far from perfect, but I couldn't find a way to scrap DI.fm's original grouping. Suit yourself modifying them. What didn't fall into any of your groups, will fall into the `('else', '')` group. We will get the `channel_name` for each channel, which is the string we will use to build the actual stream URLs.

In [10]:
groups = {
    ('house', 'deep', 'disco'): [],
    ('chill', 'lounge', 'down', 'lofi', 'ambient'): [],
    ('club', 'hits'): [],
    ('trance', ): [],
    ('classic', 'hits'): [],
    ('vocal', ): [],
    ('tech', 'minimal', 'hard'): [],
    ('jungle', 'dnb', 'drum', 'beat'): [],
    ('oldschool', 'classic'): [],
    ('psy', 'progressive'): [],
    ('oldschool', 'classic'): [],
    ('indie', 'future'): [],
    ('drum', 'break'): [],
    ('electro', 'edm'): [],
    ('else',): []
}

headers = { # DI.fm ignores non-standard clients. So we say we are Chrome
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
}

channels = requests.get(ALL_CHANNELS, headers=headers)
for channel in channels.json():
    found = False
    channel_url  = CHANNEL_URL % (channel['channel_key'], KEY)
    channel_pls  = requests.get(channel_url, headers=headers)
    channel_name = channel_pls.headers['Content-Disposition'][channel_pls.headers['Content-Disposition'].find(' - '):][3:-5]
    for group in groups.keys():
        for elm in group:
            if channel['channel_key'].find(elm) > -1:
                groups[group].append((channel['channel_key'], channel_name))
                found = True
                break
        if found:
            break
    if not found:
        groups[('else',)].append((channel['channel_key'], channel_name))

Now, we get the actual URL for each stream. We know the `channel_name`, now we just need to define the right url, by setting `kbps` to either `64`, `128` or `320`. And with that, build the playlist for each group.

In [11]:
kbps = 320
stream_urls = {
    320: 'http://prem1.di.fm:80/%s_hi?%s',
    128: 'http://prem1.di.fm:80/%s?%s',
    64 : 'http://prem1.di.fm:80/%s_aac?%s',
}

header = """[playlist]
NumberOfEntries=%i
"""
entry = """File%i=""" + stream_urls[kbps] + """
Title1=DI.FM - %s
Length%i=0
"""
for group_name, group_members in groups.items():
    # print(group_name)
    if len(group_name) > 1:
        group_name = ' + '.join((gn.title() for gn in group_name))
    else:
        group_name = group_name[0]
    group_list = [header % len(group_members)]
    channel_num = 0
    for channel_key, channel_name in group_members:
        # print(' - ', channel_name)
        group_list.append(entry % (channel_num, channel_key, KEY, channel_name, channel_num))
        channel_num+=1
    open(os.path.join('pls', 'DI_fm - ' + group_name + '.pls'), 'w').write('\n'.join(group_list))

In [5]:
print('\n'.join(group_list))

[playlist]
NumberOfEntries=15

File0=http://prem1.di.fm:80/bassline_hi?9564836dd232258
Title1=DI.FM - Bassline
Length0=0

File1=http://prem1.di.fm:80/djmixes_hi?9564836dd232258
Title1=DI.FM - DJ Mixes
Length1=0

File2=http://prem1.di.fm:80/dub_hi?9564836dd232258
Title1=DI.FM - Dub
Length2=0

File3=http://prem1.di.fm:80/dubstep_hi?9564836dd232258
Title1=DI.FM - Dubstep
Length3=0

File4=http://prem1.di.fm:80/eurodance_hi?9564836dd232258
Title1=DI.FM - EuroDance
Length4=0

File5=http://prem1.di.fm:80/gabber_hi?9564836dd232258
Title1=DI.FM - Gabber
Length5=0

File6=http://prem1.di.fm:80/glitchhop_hi?9564836dd232258
Title1=DI.FM - Glitch Hop
Length6=0

File7=http://prem1.di.fm:80/handsup_hi?9564836dd232258
Title1=DI.FM - Hands Up
Length7=0

File8=http://prem1.di.fm:80/liquiddubstep_hi?9564836dd232258
Title1=DI.FM - Liquid Dubstep
Length8=0

File9=http://prem1.di.fm:80/liquidtrap_hi?9564836dd232258
Title1=DI.FM - Liquid Trap
Length9=0

File10=http://prem1.di.fm:80/nightcore_hi?9564836dd23225